In [1]:
%matplotlib qt
import numpy as np 
import matplotlib.pyplot as plt
from spec_im import gui_fname, plot_pl_summary, plot_pl_3d_summary, plot_si_bands, plot_bss_results, plot_decomp_results
from spec_im import SpectralImage, PLSpectralImage, PL3DSpectralImage
import math
import hyperspy.api as hs
hs.preferences.gui(toolkit="traitsui")

# pl_3d_plot
Plots a 3D spectral image. 

## 1. Load file

In [2]:
fname = gui_fname()
# fname = 'D:/Chris/BBOD_Share/uv_microscope/190430 confocal GaN pyramid uvpl/190424_163834_oo_asi_hyperspec_3d_scan.h5'

In [3]:
si = PL3DSpectralImage(fname=fname)
#si.plot_spec()

Load from C:/Users/Chris Chen/Google Drive/CoSES/Data/UV PL/190424_163834_oo_asi_hyperspec_3d_scan.h5 complete.
5 x 41 x 41 spatial x 1044 spectral points


In [ ]:
plt.figure()
si = si[200:950]
si.plot_spec()

In [4]:
#plt.figure()
si.set_background(lims=(700, 900))
#si.plot_spec()

-8.100746268656621


In [5]:
si = si[356:600]
plt.figure()
si.plot_spec()

## 2. Visualize file

In [ ]:
plot_pl_3d_summary(si, num_rows=3);

In [6]:
s = si.to_signal()

In [7]:
s.plot()

In [ ]:
plt.figure()
si.plot(show_cbar=True, cbar_orientation='vertical', cbar_position='right', show_axes=True, show_scalebar=False)

In [8]:
s2d = s.as_signal2D((0,1))

In [9]:
s2d.plot()

In [ ]:
s1d = s2d.as_signal1D(3)

In [ ]:
s1d = s2d.transpose(navigation_axes=[2,3,1])

In [ ]:
s1d

In [ ]:
.plot()

In [ ]:
s1d.plot()

In [ ]:
s1d.decomposition(algorithm='svd')
s1d.plot_explained_variance_ratio()

In [ ]:
s1d.plot_decomposition_results()

In [ ]:
s2d

## 3. Hyperspy decomposition

In [10]:
# perform principal component analysis, look at the explained variance
s.decomposition(algorithm='svd')
s.plot_explained_variance_ratio()

In [11]:
s.plot_decomposition_results()

Traceback (most recent call last):
  File "C:\Miniconda3\envs\hyperspy\lib\site-packages\matplotlib\cbook\__init__.py", line 215, in process
    func(*args, **kwargs)
  File "C:\Miniconda3\envs\hyperspy\lib\site-packages\hyperspy\drawing\utils.py", line 171, in function_wrapper
    function()
  File "C:\Miniconda3\envs\hyperspy\lib\site-packages\hyperspy\drawing\signal1d.py", line 144, in _on_close
    super(Signal1DFigure, self)._on_close()
  File "C:\Miniconda3\envs\hyperspy\lib\site-packages\hyperspy\drawing\figure.py", line 103, in _on_close
    self.events.closed.trigger(obj=self)
  File "<string>", line 4, in trigger
  File "C:\Miniconda3\envs\hyperspy\lib\site-packages\hyperspy\events.py", line 402, in trigger
    function(**{kw: kwargs.get(kw, None) for kw in kwsl})
  File "C:\Miniconda3\envs\hyperspy\lib\site-packages\hyperspy\signal.py", line 2078, in <lambda>
    lambda: self.events.data_changed.disconnect(self.update_plot),
  File "C:\Miniconda3\envs\hyperspy\lib\site-packa

In [ ]:
COMPS = 4

In [ ]:
# take a look at orthogonalized 
plot_bss_results(s,si,title='BSS',cmap='viridis',fig_rows=min((5, COMPS)),components=COMPS,max_iter=2000)

In [12]:
sc = s.get_decomposition_model(components=(0,1,2,3,12,23))
sc.plot()

In [13]:
(s-sc).plot()

In [14]:
sc2d = sc.as_signal2D((0,1))

In [15]:
sc2d.plot()

In [ ]:
plot_decomp_results(nzsc, asdf, title='NMF', cmap='viridis', fig_rows=4, algorithm='nmf', output_dimension=8)

## A1. sklearn based native decomposition

In [ ]:
si.decomposition(algorithm='svd', output_dimension=0.9999)

In [ ]:
si.plot_explained_variance_ratio()

In [ ]:
si.blind_source_separation(number_of_components=10, max_iter=2000)

In [ ]:
ii = 0
slice_list = si.get_slice_list()
loadings = si.get_bss_loadings()
factors = si.get_bss_factors()
num_factors = factors.shape[0]

num_rows = 5
num_figures = math.ceil(float(num_factors)/num_rows)
print(loadings.shape, factors.shape, si.spec_x_array.shape)
(nz, ny, nx, nf) = np.shape(si.spec_im)
num_cols = 3 + nz
show_scalebar=False

for jj in range(num_figures):
    plt.figure()
    for kk in range(num_rows):
        if jj*num_rows + kk >= num_factors:  
            break
        index = kk + jj*num_rows
        # print(num_rows, num_cols, 3*kk + 1)
        for ii in range(nz):
            plt.subplot(num_rows, num_cols, num_cols*kk + 1 + ii)
            if kk == 0:
                z_pos = 'z = %0.2f $\mu$m' % ((si.z_array[ii]-si.z_array[0])*1e3)
            else:
                z_pos = ''
                
            if kk==0 and ii==0:
                show_scalebar=True
            else:
                show_scalebar=False
            
            si._plot(loadings[index, ii, :, :], cbar_orientation='vertical', cbar_position='right',
                     title='%s' % z_pos, show_scalebar=show_scalebar)
            # print(num_rows, num_cols, (3*kk + 2, 3*kk+3))
        plt.subplot(num_rows, num_cols, (num_cols*kk + nz + 1, num_cols*kk + nz + 2))
        si._plot_spec(factors[index,:])
        plt.title('%d' % index)
        

In [ ]:
slice_list = si.get_slice_list()
plt.figure()
si.get_slice(slice_list[1])[330:360].plot()

In [ ]:
mask = si.get_slice(slice_list[2])[330:360].spec_im.sum(axis=-1) < 7.95e4
print(mask.shape)
plt.figure()
plt.imshow(mask)
si.apply_mask(mask)

In [ ]:
# mask2 = np.empty(np.shape(si.spec_im.sum(axis=-1)))
# mask2[kk in range(np.size(si.z_array)), :, :, :] = mask
# #si.apply_mask(mask2)
# print('mask size', np.size(mask2), 'nonzero values in mask', np.count_nonzero(mask2.flatten()), '', np.size(si.spec_im.sum(axis=-1)))

In [ ]:
si.decomposition(algorithm='svd', output_dimension=0.9999)
si.plot_explained_variance_ratio()

In [ ]:
si.blind_source_separation(number_of_components=10)

In [ ]:
ii = 0
slice_list = si.get_slice_list()
loadings = si.get_bss_loadings()
factors = si.get_bss_factors()
num_factors = factors.shape[0]

num_rows = 5
num_figures = math.ceil(float(num_factors)/num_rows)
print(loadings.shape, factors.shape, si.spec_x_array.shape)
(nz, ny, nx, nf) = np.shape(si.spec_im)
num_cols = 3 + nz
show_scalebar=False

for jj in range(num_figures):
    plt.figure()
    for kk in range(num_rows):
        if jj*num_rows + kk >= num_factors:  
            break
        index = kk + jj*num_rows
        # print(num_rows, num_cols, 3*kk + 1)
        for ii in range(nz):
            plt.subplot(num_rows, num_cols, num_cols*kk + 1 + ii)
            if kk == 0:
                z_pos = 'z = %0.2f $\mu$m' % ((si.z_array[ii]-si.z_array[0])*1e3)
            else:
                z_pos = ''
                
            if kk==0 and ii==0:
                show_scalebar=True
            else:
                show_scalebar=False
            
            si._plot(loadings[index, ii, :, :], cbar_orientation='vertical', cbar_position='right',
                     title='%s' % z_pos, show_scalebar=show_scalebar)
            # print(num_rows, num_cols, (3*kk + 2, 3*kk+3))
        plt.subplot(num_rows, num_cols, (num_cols*kk + nz + 1, num_cols*kk + nz + 2))
        si._plot_spec(factors[index,:])
        plt.title('%d' % index)
        

In [ ]:
sc = s.get_decomposition_model(components=10)
sc.plot()

In [16]:
print('asdf')

asdf


In [ ]:
sc = s.get_decomposition_model(components=4)
sc.plot()

In [ ]:
from hyperspy.signals import Signal2D